# Index file visualization

This notebook shows an easy way to represent the In Situ data positions using the index files.<br>
For this visualization of a sample <i>index_latest.txt</i> dataset of the Copernicus Marine Environment Monitoring Service.

In [11]:
indexfile = "datafiles/index_latest.txt"

## Import packages

We use the two packages:
* <a href="https://github.com/python-visualization/folium">folium</a> for the visualization and
* <a href="http://www.numpy.org/">numpy</a> for the data reading / processing.

In [12]:
import numpy as np
import folium

# Selection criteria

Here we show how the data positions can be selected prior to the plot.<br>
The example explains the selection by data providers, but that can be generalised to other properties.

## Provider

The list dataprovider contains the name of the providers we want to keep for the plot.

In [13]:
dataproviderlist = ['IEO', 'INSTITUTO ESPANOL DE OCEANOGRAFIA', 'SOCIB']

Here we could also add something for the time or space domain.

## Load and prepare data

Since the <i>index_latest.txt</i> is a formatted file, we use the numpy function <a href="http://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html">genfromtxt</a> to extract the data from the document.

In [14]:
dataindex = np.genfromtxt(indexfile, skip_header=6, unpack=True, delimiter=',', dtype=None, \
              names=['catalog_id', 'file_name', 'geospatial_lat_min', 'geospatial_lat_max',
                     'geospatial_lon_min', 'geospatial_lon_max',
                     'time_coverage_start', 'time_coverage_end', 
                     'provider', 'date_update', 'data_mode', 'parameters'])

For each data file, the positions are defined as a bounding box. <br> 
To define the position shown on the map, we use the mean of the stored <i>geospatial_lat/lon_min/max</i> for each dataset.

In [15]:
lon_min = dataindex['geospatial_lon_min']
lon_max = dataindex['geospatial_lon_max']
lat_min = dataindex['geospatial_lat_min']
lat_max = dataindex['geospatial_lat_max']
lonmean, latmean = 0.5*(lon_min + lon_max), 0.5*(lat_min + lat_max)

# Select by data provider

We create a list of indices corresponding to the entries with a provider belonging to the list specified at the beginning.

In [16]:
indexlist = []
for np, provider in enumerate(dataindex['provider']):
    matching = [s for s in dataproviderlist if s in provider]
    if matching:
        indexlist.append(np)

Could do intersection of the list, but for that we need to specify the provider name as specified in the index file.

## Visualization

Finally, we create the map object.<br>
The map is centered on the Mediterranean Sea, this has to be changed according to the region of interest and the index file we consider.

In [17]:
map = folium.Map(location=[39.5, 2], zoom_start=8)
cntr = 0
for i in indexlist:
    curr_data = dataindex[i]
    link = curr_data[1]
    
    last_idx_slash = link.rfind('/')
    
    ncdf_file_name = link[last_idx_slash+1::]
    folium.Marker( location = [latmean[i], lonmean[i]], popup=ncdf_file_name).add_to(map)

Finally we can create the map where we will see the locations of the platforms as well as the corresponding file names.

In [18]:
map